# Image scraper for Sanrio webshop

This script is an image scraper for the online shop of Sanrio to collect data of authentic plushies.

Import all necessary models to make the image scraper work and save all the contents

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
from builtins import open, bytes
import uuid
import time

The scraper is into a function so it can be executed multiple times by just referencing it. The function accept a URL with the parameter 'url' <br>
1. A response to the URL is made, to which the page HTML source is parsed in test
2. The script finds the titles of the listings in the HTML page source, which is in the aria-label
3. A folder where the images go is refenced in a variable. If the folder does not exist, it will be created first
4. Using a 'for loop', the images are downloaded from their source URLs and saved in the aforementioned folder. To avoid the possibility of overloading the server, the script pauses 3 seconds for every 10 seconds. The images are retrieved and saved by creating the file first and copying the source data from the images in it as binary data, which is used for non-text files
<br> <br> 
The images are named according to the title of the listing of the plush, which is from the aria-label. To prevent the images from overridding eachother, an incremented number is added at the end of the filename. The script also checks if the source URL starts with 'http' to download the images. If not the case, a protocol indicator is added at the start of the URL.

In [2]:
def download_images(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    protocol_indicator = "https:"

    a = soup.find_all("a", {"aria-label": True})

    folder_name = 'authentic'
    if not os.path.exists(folder_name):
            os.makedirs(folder_name)

    for i, aria_labels in enumerate (a):
        if i % 10 == 0:
            time.sleep(3)
        imgs = aria_labels.find_all("img", {"src": True})
        for i, img in enumerate (imgs):
            src = img["src"]
            
            if not src.startswith("http"):
                src = protocol_indicator + src
            response = requests.get(src)
            file_name = aria_labels["aria-label"]
        
            with open(f"{folder_name}/{file_name}_{i}.jpg", 'wb') as f:
                f.write(response.content)

The function is called with the URL in the parameter, to which all the images are downloaded in the folder. As there are multiple pages of listings on the site, the function would have to be called for each one

In [4]:
download_images("https://www.sanrio.com/collections/plush")
download_images("https://www.sanrio.com/collections/plush?page=2")
download_images("https://www.sanrio.com/collections/plush?page=3")
download_images("https://www.sanrio.com/collections/plush?page=4")